# 1. 모듈, 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import time
import numpy as np
import re

In [2]:
BillDf = pd.read_csv("./21대 국회 의안정보 new.csv", encoding='utf-8-sig') #200824업데이트
BillDf.head()

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
0,2103236,계,유통산업발전법 일부개정법률안(박재호의원 등 10인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,['제안이유 및 주요내용현행법은 대ㆍ중소유통업의 상생발전과 지역상권의 활성화를 도모...,[],"['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김정호(더불..."
1,2103235,계,도로교통법 일부개정법률안(황운하의원 등 13인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,['제안이유 및 주요내용현행법은 자동차등이 정지하고 있는 경우 등을 제외하고는 운전...,[],"['황운하(더불어민주당/黃雲夏)', '김경만(더불어민주당/金京萬)', '김경협(더불..."
2,2103234,계,철도산업발전기본법 일부개정법률안(이은주의원 등 30인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,"['제안이유 및 주요내용정부는 「노인복지법」,「장애인복지법」 등에 따라 65세 이상...",[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
3,2103233,계,도시철도법 일부개정법률안(이은주의원 등 29인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,"['제안이유정부는 「노인복지법」,「장애인복지법」 등에 따라 65세 이상 노인·장애인...",[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
4,2103232,계,대중교통의 육성 및 이용촉진에 관한 법률 일부개정법률안(이은주의원 등 29인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,"['제안이유최근 코로나19 바이러스의 확산에 따라 외출 자제 및 재택근무 권고, 학...",[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."


# 2. 전처리

#### ①  [의안번호] ZZ로 시작하는 행 삭제

In [3]:
indexlist=[]

for k in range(len(BillDf)):
    if 'ZZ' in str(BillDf['의안번호'][k]):
        indexlist.append(k)


indexlist
BillDf = BillDf.drop(indexlist)        

In [4]:
# 인덱스 리셋

BillDf.reset_index(drop=False, inplace=True)

#### ② [제안이유] 특수문자 및 '제안이유 및 주요내용', '제안이유', '주요내용' 문자열 삭제

In [5]:
def clean_reason(df_col): 
    text = df_col[2:-2].replace('제안이유 및 주요내용','').replace('대안의 제안이유 및 주요내용','').replace('대안의 주요내용','').replace('대안의 제안이유','').replace('제안이유','').replace('주요내용','')
    return re.sub('[-=.#/:$}·,■?]', ' ', text)

In [6]:
BillDf['제안이유'] = BillDf['제안이유'].apply(lambda x: clean_reason(x))

In [7]:
BillDf.head(10)

,index,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
0,0,2103236,계,유통산업발전법 일부개정법률안(박재호의원 등 10인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 대ㆍ중소유통업의 상생발전과 지역상권의 활성화를 도모하기 위하여 대규모점포와...,[],"['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김정호(더불..."
1,1,2103235,계,도로교통법 일부개정법률안(황운하의원 등 13인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 자동차등이 정지하고 있는 경우 등을 제외하고는 운전 중에 영상표시장치를 시...,[],"['황운하(더불어민주당/黃雲夏)', '김경만(더불어민주당/金京萬)', '김경협(더불..."
2,2,2103234,계,철도산업발전기본법 일부개정법률안(이은주의원 등 30인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,정부는 「노인복지법」 「장애인복지법」 등에 따라 65세 이상 노인 장애인 등에 대하...,[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
3,3,2103233,계,도시철도법 일부개정법률안(이은주의원 등 29인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,정부는 「노인복지법」 「장애인복지법」 등에 따라 65세 이상 노인 장애인 등에 대하...,[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
4,4,2103232,계,대중교통의 육성 및 이용촉진에 관한 법률 일부개정법률안(이은주의원 등 29인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,최근 코로나19 바이러스의 확산에 따라 외출 자제 및 재택근무 권고 학교 개학 연...,[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
5,5,2103231,계,국정감사 및 조사에 관한 법률 일부개정법률안(박재호의원 등 10인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법에 따르면 위원회 소위원회 또는 반은 감사 또는 조사를 위하여 필요한 자료를...,[],"['박재호(더불어민주당/朴在昊)', '김수흥(더불어민주당/金洙興)', '김정호(더불..."
6,6,2103230,계,도시재생 활성화 및 지원에 관한 특별법 일부개정법률안(박재호의원 등 10인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,지역의 도시재생 역량 제고를 위해 각 대학에서 도시재생 분야를 전공한 전문인력의 확...,[],"['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김수흥(더불..."
7,7,2103229,계,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안(박재호의원 ...,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,「해양산업클러스터의 지정 및 육성 등에 관한 특별법」은 해양산업클러스터 개발을 위하...,[],"['박재호(더불어민주당/朴在昊)', '김승남(더불어민주당/金承南)', '김윤덕(더불..."
8,8,2103228,계,자본시장과 금융투자업에 관한 법률 일부개정법률안(박용진의원 등 11인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,공매도는 주가하락 예상 시 차입한 주식으로 매도 후 결제하고 실제로 주가하락 시 ...,[],"['박용진(더불어민주당/朴用鎭)', '기동민(더불어민주당/奇東旻)', '김경만(더불..."
9,9,2103227,계,유료도로법 일부개정법률안(박재호의원 등 10인),의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 유료도로관리청 또는 유료도로관리권자는 해당 유료도로를 통행한 자가 통행료를...,[],"['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김수흥(더불..."


#### ③ [의안명]뒤 대표제안자 포함 괄호 삭제

In [8]:
for i in range(len(BillDf)):
    BillDf['의안명'][i] = BillDf['의안명'][i].split('(')[0]

<ipython-input-8-3436be3a82c0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BillDf['의안명'][i] = BillDf['의안명'][i].split('(')[0]


In [9]:
BillDf.head(10)

,index,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
0,0,2103236,계,유통산업발전법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 대ㆍ중소유통업의 상생발전과 지역상권의 활성화를 도모하기 위하여 대규모점포와...,[],"['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김정호(더불..."
1,1,2103235,계,도로교통법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 자동차등이 정지하고 있는 경우 등을 제외하고는 운전 중에 영상표시장치를 시...,[],"['황운하(더불어민주당/黃雲夏)', '김경만(더불어민주당/金京萬)', '김경협(더불..."
2,2,2103234,계,철도산업발전기본법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,정부는 「노인복지법」 「장애인복지법」 등에 따라 65세 이상 노인 장애인 등에 대하...,[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
3,3,2103233,계,도시철도법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,정부는 「노인복지법」 「장애인복지법」 등에 따라 65세 이상 노인 장애인 등에 대하...,[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
4,4,2103232,계,대중교통의 육성 및 이용촉진에 관한 법률 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,최근 코로나19 바이러스의 확산에 따라 외출 자제 및 재택근무 권고 학교 개학 연...,[],"['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당/姜恩..."
5,5,2103231,계,국정감사 및 조사에 관한 법률 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법에 따르면 위원회 소위원회 또는 반은 감사 또는 조사를 위하여 필요한 자료를...,[],"['박재호(더불어민주당/朴在昊)', '김수흥(더불어민주당/金洙興)', '김정호(더불..."
6,6,2103230,계,도시재생 활성화 및 지원에 관한 특별법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,지역의 도시재생 역량 제고를 위해 각 대학에서 도시재생 분야를 전공한 전문인력의 확...,[],"['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김수흥(더불..."
7,7,2103229,계,공익사업을 위한 토지 등의 취득 및 보상에 관한 법률 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,「해양산업클러스터의 지정 및 육성 등에 관한 특별법」은 해양산업클러스터 개발을 위하...,[],"['박재호(더불어민주당/朴在昊)', '김승남(더불어민주당/金承南)', '김윤덕(더불..."
8,8,2103228,계,자본시장과 금융투자업에 관한 법률 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,공매도는 주가하락 예상 시 차입한 주식으로 매도 후 결제하고 실제로 주가하락 시 ...,[],"['박용진(더불어민주당/朴用鎭)', '기동민(더불어민주당/奇東旻)', '김경만(더불..."
9,9,2103227,계,유료도로법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 유료도로관리청 또는 유료도로관리권자는 해당 유료도로를 통행한 자가 통행료를...,[],"['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김수흥(더불..."


#### ④ [제안자구분] -> 의원, 의장, 위원장 데이터만 남기고 삭제

In [10]:
indexlist=[]

for k in range(len(BillDf)):
    if BillDf['제안자구분'][k]!='의원' and BillDf['제안자구분'][k]!='의장' and BillDf['제안자구분'][k]!='위원장':
        indexlist.append(k)



BillDf = BillDf.drop(indexlist)   

In [11]:
# 인덱스 리셋

BillDf.reset_index(drop=False, inplace=True)

- 문화체육관광위원회 -> 도종환(더불어민주당)
- 국토교통위원회 -> 진선미(더불어민주당)
- 보건복지위원회 -> 한정애(더불어민주당)
- 법제사법위원회 -> 윤호중(더불어민주당)
- 국회상임위원회 위원 정수에 관한 규칙 개정 특별 위원회 -> 김영진(더불어민주당) 


#### ⑤ [제안자] 칼럼 스플릿, 제안 정당만 따와서 리스트로 변환 (중복제거) -> [제안정당] 칼럼 생성

In [12]:
# [제안자]칼럼 콤마 기준 스플릿

BillDf['제안자'] = BillDf['제안자'].str.split(',')

In [13]:
BillDf['제안자'] = BillDf['제안자'].fillna("a") #object of type 'float' has no len() 오류 해결

list1 = []

for i in range(len(BillDf)):
    
    list2 = []
    
    if BillDf['제안자'][i] != "a":
        
        for j in range(len(BillDf['제안자'][i])):
            
            list2.append(BillDf['제안자'][i][j].split('(')[-1].split('/')[0])
            
        list1.append(list(set(list2)))
    
    else:
        list1.append(np.NaN)
        
BillDf['제안정당'] = list1  


In [14]:
BillDf

,level_0,index,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,제안정당
0,0,0,2103236,계,유통산업발전법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 대ㆍ중소유통업의 상생발전과 지역상권의 활성화를 도모하기 위하여 대규모점포와...,[],"[['박재호(더불어민주당/朴在昊)', '김경협(더불어민주당/金炅俠)', '김정호...",[더불어민주당]
1,1,1,2103235,계,도로교통법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,현행법은 자동차등이 정지하고 있는 경우 등을 제외하고는 운전 중에 영상표시장치를 시...,[],"[['황운하(더불어민주당/黃雲夏)', '김경만(더불어민주당/金京萬)', '김경협...",[더불어민주당]
2,2,2,2103234,계,철도산업발전기본법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,정부는 「노인복지법」 「장애인복지법」 등에 따라 65세 이상 노인 장애인 등에 대하...,[],"[['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당...","[정의당, 미래통합당, 열린민주당, 무소속, 더불어민주당]"
3,3,3,2103233,계,도시철도법 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,정부는 「노인복지법」 「장애인복지법」 등에 따라 65세 이상 노인 장애인 등에 대하...,[],"[['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당...","[정의당, 미래통합당, 열린민주당, 무소속, 더불어민주당]"
4,4,4,2103232,계,대중교통의 육성 및 이용촉진에 관한 법률 일부개정법률안,의원,2020-08-24,NaN,NaN,접수,NaN,제21대 (2020~2024) 제381회,최근 코로나19 바이러스의 확산에 따라 외출 자제 및 재택근무 권고 학교 개학 연...,[],"[['이은주(정의당/李恩周)', '강민정(열린민주당/姜旼姃)', '강은미(정의당...","[정의당, 미래통합당, 열린민주당, 무소속, 더불어민주당]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,4227,4249,2100005,계,고등교육법 일부개정법률안,의원,2020-06-01,NaN,NaN,소관위접수,의안원문\r\n비용추계서 미첨부 사유서,제21대 (2020~2024) 제378회,전대미문의 코로나 19 감염병의 장기간 창궐로 대학들은 감염병 예방을 위해 온라인 ...,교육위원회,"[['이종배(미래통합당/李鍾培)', '강기윤(미래통합당/姜起潤)', '강대식(미...",[미래통합당]
3987,4228,4250,2100004,계,감염병의 예방 및 관리에 관한 법률 일부개정법률안,의원,2020-06-01,NaN,NaN,소관위심사,의안원문\r\n비용추계서 미첨부 사유서,제21대 (2020~2024) 제378회,현행법에 따르면 감염병 확산방지를 위한 격리시설의 설치 운영 감염병 환자를 진료한...,보건복지위원회,"[['이종배(미래통합당/李鍾培)', '강기윤(미래통합당/姜起潤)', '강대식(미...",[미래통합당]
3988,4229,4251,2100003,계,장애인활동 지원에 관한 법률 일부개정법률안,의원,2020-06-01,NaN,NaN,소관위심사,의안원문\r\n비용추계서 미첨부 사유서,제21대 (2020~2024) 제378회,현행법은 「노인장기요양보험법」에 따른 노인등을 활동지원급여 신청자격에서 제외하고 있...,보건복지위원회,"[['장제원(미래통합당/張濟元)', '권성동(무소속/權性東)', '김정재(미래통...","[무소속, 미래통합당]"
3989,4230,4252,2100002,계,정부조직법 일부개정법률안,의원,2020-06-01,NaN,NaN,소관위접수,의안원문\r\n비용추계서,제21대 (2020~2024) 제378회,코로나바이러스감염증 19(COVID 19) 대유행 이후 국가 감염병 대응 역량을 강...,행정안전위원회,"[['신현영(더불어민주당/申賢榮)', '고용진(더불어민주당/高榕禛)', '김경만...","[무소속, 더불어민주당]"


#### ⑥ 제안자구분이 '위원장'인 경우 제안정당->더불어민주당으로 채우기

In [15]:
BillDf[BillDf['제안자구분']=='위원장'].head(20)

,level_0,index,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,제안정당
564,585,587,2102651,처,국민체육진흥법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,체육계 성폭력 등 폭력에 대한 예방조치 및 가해자에 대한 강화된 제재 근거의 마련 ...,문화체육관광위원회,[[]],[[]]
565,586,588,2102650,처,주택법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,가 노후ㆍ불량건축물이 밀집한 지역을 정비하고 주택 등 건축물을 개량 또는 건설하는...,국토교통위원회,[[]],[[]]
566,587,589,2102649,처,민간임대주택에 관한 특별법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,대안의 가 단기민간임대주택 장기일반민간임대주택 중 아파트로 등록하는 매입임대주택...,국토교통위원회,[[]],[[]]
567,588,590,2102648,처,감염병의 예방 및 관리에 관한 법률 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,정부이송,의안원문,제21대 (2020~2024) 제380회,코로나19는 높은 전파력을 보이고 있어 집단감염 예방을 위해서는 마스크 착용 등 방...,보건복지위원회,[[]],[[]]
715,736,738,2102500,처,주택임대차보호법 일부개정법률안,위원장,2020-07-29,2020-07-30,원안가결,정부이송,의안원문,제21대 (2020~2024) 제380회,주택시장의 불안정 속에 전세에서 월세로의 전환이 빨라지고 주택 임대료가 상승함에 따...,법제사법위원회,[[]],[[]]
1538,1581,1587,2102651,처,국민체육진흥법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,체육계 성폭력 등 폭력에 대한 예방조치 및 가해자에 대한 강화된 제재 근거의 마련 ...,문화체육관광위원회,[[]],[[]]
1539,1582,1588,2102650,처,주택법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,가 노후ㆍ불량건축물이 밀집한 지역을 정비하고 주택 등 건축물을 개량 또는 건설하는...,국토교통위원회,[[]],[[]]
1540,1583,1589,2102649,처,민간임대주택에 관한 특별법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,대안의 가 단기민간임대주택 장기일반민간임대주택 중 아파트로 등록하는 매입임대주택...,국토교통위원회,[[]],[[]]
1541,1584,1590,2102648,처,감염병의 예방 및 관리에 관한 법률 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,정부이송,의안원문,제21대 (2020~2024) 제380회,코로나19는 높은 전파력을 보이고 있어 집단감염 예방을 위해서는 마스크 착용 등 방...,보건복지위원회,[[]],[[]]
1689,1732,1738,2102500,처,주택임대차보호법 일부개정법률안,위원장,2020-07-29,2020-07-30,원안가결,정부이송,의안원문,제21대 (2020~2024) 제380회,주택시장의 불안정 속에 전세에서 월세로의 전환이 빨라지고 주택 임대료가 상승함에 따...,법제사법위원회,[[]],[[]]


In [16]:
for i in range(len(BillDf)):
    if BillDf['제안자구분'][i]=='위원장':
        BillDf['제안정당'][i]=[]
        BillDf['제안정당'][i].append('더불어민주당')

<ipython-input-16-ff7cc8bc163b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BillDf['제안정당'][i]=[]


In [17]:
BillDf[BillDf['제안자구분']=='위원장'].head(5)

,level_0,index,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,제안정당
564,585,587,2102651,처,국민체육진흥법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,체육계 성폭력 등 폭력에 대한 예방조치 및 가해자에 대한 강화된 제재 근거의 마련 ...,문화체육관광위원회,[[]],[더불어민주당]
565,586,588,2102650,처,주택법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,가 노후ㆍ불량건축물이 밀집한 지역을 정비하고 주택 등 건축물을 개량 또는 건설하는...,국토교통위원회,[[]],[더불어민주당]
566,587,589,2102649,처,민간임대주택에 관한 특별법 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,공포,의안원문,제21대 (2020~2024) 제380회,대안의 가 단기민간임대주택 장기일반민간임대주택 중 아파트로 등록하는 매입임대주택...,국토교통위원회,[[]],[더불어민주당]
567,588,590,2102648,처,감염병의 예방 및 관리에 관한 법률 일부개정법률안,위원장,2020-08-03,2020-08-04,원안가결,정부이송,의안원문,제21대 (2020~2024) 제380회,코로나19는 높은 전파력을 보이고 있어 집단감염 예방을 위해서는 마스크 착용 등 방...,보건복지위원회,[[]],[더불어민주당]
715,736,738,2102500,처,주택임대차보호법 일부개정법률안,위원장,2020-07-29,2020-07-30,원안가결,정부이송,의안원문,제21대 (2020~2024) 제380회,주택시장의 불안정 속에 전세에서 월세로의 전환이 빨라지고 주택 임대료가 상승함에 따...,법제사법위원회,[[]],[더불어민주당]


#### ⑦ 당별 의안명+제안이유 텍스트 리스트에 추가

In [18]:
# 제안 정당 개수 확인

a = []
for i in range(len(BillDf)):
    a.append(len(BillDf['제안정당'][i]))
    
a = set(a)
a = list(a)

a

[1, 2, 3, 4, 5, 6, 8]

In [19]:
# 정당 리스트 확인

partylist = []

for i in range(len(BillDf)):
    for j in range(len(BillDf['제안정당'][i])):
        if BillDf['제안정당'][i][j] not in partylist:
            partylist.append(BillDf['제안정당'][i][j])
partylist

['더불어민주당',
 '정의당',
 '미래통합당',
 '열린민주당',
 '무소속',
 '국민의당',
 '기본소득당',
 " ''",
 '시대전환',
 '[]']

In [20]:
# 당별 텍스트 리스트 생성

DM_text = []
MT_text = []
MS_text = []
JE_text = []
YM_text = []
KM_text = []
KS_text = []
SJ_text = []

In [21]:
for i in range(len(BillDf)):
    for j in range(len(BillDf['제안정당'][i])):
        if '더불어민주당' in BillDf['제안정당'][i][j]:
            DM_text.append(BillDf['의안명'][i])
            DM_text.append(BillDf['제안이유'][i])
        elif '미래통합당' in BillDf['제안정당'][i][j]:
            MT_text.append(BillDf['의안명'][i])
            MT_text.append(BillDf['제안이유'][i])
        elif '무소속' in BillDf['제안정당'][i][j]:
            MS_text.append(BillDf['의안명'][i])
            MS_text.append(BillDf['제안이유'][i])
        elif '정의당' in BillDf['제안정당'][i][j]:
            JE_text.append(BillDf['의안명'][i])
            JE_text.append(BillDf['제안이유'][i])
        elif '열린민주당' in BillDf['제안정당'][i][j]:
            YM_text.append(BillDf['의안명'][i])
            YM_text.append(BillDf['제안이유'][i])
        elif '국민의당' in BillDf['제안정당'][i][j]:
            KM_text.append(BillDf['의안명'][i])
            KM_text.append(BillDf['제안이유'][i])
        elif '기본소득당' in BillDf['제안정당'][i][j]:
            KS_text.append(BillDf['의안명'][i])
            KS_text.append(BillDf['제안이유'][i])
        elif '시대전환' in BillDf['제안정당'][i][j]:
            SJ_text.append(BillDf['의안명'][i])
            SJ_text.append(BillDf['제안이유'][i])
        

# 3. Topic Modeling 함수제작

In [22]:
import matplotlib.pyplot as plt
from string import punctuation
from konlpy.tag import Kkma
import time
import datetime
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim

ModuleNotFoundError: No module named 'pyLDAvis'

In [ ]:
stopword = pd.read_csv('./한국어불용어100.txt', sep = "\t")
stopword_list = list(stopword['이'])
stopword_list

#### ① 텍스트 전처리

In [ ]:
## 특수문자 제거 & 명사추출

def remove_special(textlist):
    
    kkma = Kkma()
    nounlist = []
    
    for i in range(len(textlist)):
        n = re.sub('[-=.#/:$}·,■?]','',textlist[i])
        k = re.sub('[0-9]','',n)
        nounlist.append(kkma.nouns(k))
        
    return nounlist


## 특정 문자열 삭제 & 한글자짜리는 삭제

def remove_string(nounlist, stopword_list):
    
    cleanlist = []
    removedict = ['법률','경우','것임','대한','신설','규정','현행법','사업','지원','위원회','관리','국가','해당','발생','관련','사항','있음']
    
    for i in nounlist:
        if (i not in removedict) and (i not in stopword_list):
            cleanlist.append(i)
        
    return cleanlist
    
    
    
            
## 전체 전처리 실행

def Preproc(textlist):
    
    nounlist = remove_special(textlist)
    cleanlist = remove_string(nounlist, stopword_list)
    
    return cleanlist


#### ② 딕셔너리 -> 말뭉치

In [ ]:
def make_corpus(cleanlist):
    
    cleanlist_ = []
    for i in cleanlist:
        if len(str(i))!=1:
            cleanlist_.append(i)
    
    dictionary = corpora.Dictionary(cleanlist_)
    corpus = [dictionary.doc2bow(text) for text in cleanlist_]
    
    return corpus, dictionary, cleanlist_
    

#### ③ 토픽 최적화 - Coherence Model 
토픽이 얼마나 의미론적으로 일관성 있는지 판단, 높을수록 의미론적 일관성 높음

In [ ]:
def coh_visual(corpus, dictionary, cleanlist_):
    
    coherence_values = []
    
    for i in range(2,15):
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = i, id2word = dictionary)
        coherence_model_lda = CoherenceModel(model = ldamodel, texts = cleanlist_, dictionary = dictionary, topn=10)
        coherence_lda = coherence_model_lda.get_coherence()
        coherence_values.append(coherence_lda)
    
    x = range(2,15)
    plt.plot(x, coherence_values)
    plt.xlabel('number of topics')
    plt.ylabel('coherence score')
    plt.show()

#### ④ 언어모델평가
확률 모델이 결과를 얼마나 정확하게 예측하는지 판단, 낮을수록 정확하게 예측. 

In [ ]:
def perp_visual(corpus, dictionary):
    
    perplexity_values = []
    
    for i in range(2,20):
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = i, id2word = dictionary)
        perplexity_values.append(ldamodel.log_perplexity(corpus))
    
    x = range(2,20)
    plt.plot(x, perplexity_values)
    plt.xlabel('number of topics')
    plt.ylabel('perplexity score')
    plt.show()

# 4. 더불어민주당 TopicModeling

In [ ]:
DM_clean = Preproc(DM_text)
DMcorpus, DMdictionary, DMcleanlist_ = make_corpus(DM_clean)

In [ ]:
coh_visual(DMcorpus, DMdictionary, DMcleanlist_)

In [ ]:
perp_visual(DMcorpus, DMdictionary)

=> num_topics = 6으로 결정

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(DMcorpus, num_topics=6, alpha=0.1, id2word = DMdictionary)
ldamodel.print_topics(num_words=20)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, DMcorpus, DMdictionary)
vis

DM = ldamodel.show_topic(9)
DM_ = pd.DataFrame(DM)
DM_

# 5. 미래통합당 TopicModeling

In [ ]:
MT_clean = Preproc(MT_text)
MTcorpus, MTdictionary, MTcleanlist_ = make_corpus(MT_clean)

In [ ]:
coh_visual(MTcorpus, MTdictionary, MTcleanlist_)

In [ ]:
perp_visual(MTcorpus, MTdictionary)

=> num_topics = 5로 결정

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(MTcorpus, num_topics=5, alpha=0.1, id2word = MTdictionary)
ldamodel.print_topics(num_words=20)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, MTcorpus, MTdictionary)
vis

MT = ldamodel.show_topic(9)
MT_ = pd.DataFrame(MT)
MT_

# 6. 무소속 TopicModeling

In [ ]:
MS_clean = Preproc(MS_text)
MScorpus, MSdictionary, MScleanlist_ = make_corpus(MS_clean)

In [ ]:
coh_visual(MScorpus, MSdictionary, MScleanlist_)

In [ ]:
perp_visual(MScorpus, MSdictionary)

=> number of topics = 5로 결정

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(MScorpus, num_topics=5, alpha=0.1, id2word = MSdictionary)
ldamodel.print_topics(num_words=20)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, MScorpus, MSdictionary)
vis

MS = ldamodel.show_topic(9)
MS_ = pd.DataFrame(MS)
MS_

# 7. 정의당 TopicModeling

In [ ]:
JE_clean = Preproc(JE_text)
JEcorpus, JEdictionary, JEcleanlist_ = make_corpus(JE_clean)

In [ ]:
coh_visual(JEcorpus, JEdictionary, JEcleanlist_)

In [ ]:
perp_visual(JEcorpus, JEdictionary)

=> number of topics = 9로 결정

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(JEcorpus, num_topics=9, alpha=0.1, id2word = JEdictionary)
ldamodel.print_topics(num_words=20)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, JEcorpus, JEdictionary)
vis

JE = ldamodel.show_topic(9)
JE_ = pd.DataFrame(JE)
JE_

# 8. 열린민주당 TopicModeling

In [ ]:
YM_clean = Preproc(YM_text)
YMcorpus, YMdictionary, YMcleanlist_ = make_corpus(YM_clean)

In [ ]:
coh_visual(YMcorpus, YMdictionary, YMcleanlist_)

In [ ]:
perp_visual(YMcorpus, YMdictionary)

=> number of topics = 7로 결정

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(YMcorpus, num_topics=7, alpha=0.1, id2word = YMdictionary)
ldamodel.print_topics(num_words=20)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, YMcorpus, YMdictionary)
vis

YM = ldamodel.show_topic(9)
YM_ = pd.DataFrame(YM)
YM_

# 9. 국민의당 TopicModeling

In [ ]:
KM_clean = Preproc(KM_text)
KMcorpus, KMdictionary, KMcleanlist_ = make_corpus(KM_clean)

In [ ]:
coh_visual(KMcorpus, KMdictionary, KMcleanlist_)

In [ ]:
perp_visual(KMcorpus, KMdictionary)

=> number of topics = 5로 결정

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(KMcorpus, num_topics=5, alpha=0.1, id2word = KMdictionary)
ldamodel.print_topics(num_words=20)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, KMcorpus, KMdictionary)
vis